# Synthesis with audio-dsp

This notebook covers the synthesis capabilities of `audio-dsp`:
- Subtractive Synthesis
- FM Synthesis (DX7-style)
- Chip Tune / 8-bit Sounds
- Karplus-Strong Plucked Strings
- Noise Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Metallicode/python_audio_dsp/blob/master/examples/Synthesis.ipynb)

In [ ]:
!pip install audio-dsp -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

SAMPLE_RATE = 44100

# Visualization helpers
def plot_waveform(audio, sr, title="Waveform", figsize=(12, 3), color='#2563eb'):
    """Plot a waveform with time axis."""
    plt.figure(figsize=figsize)
    time = np.arange(len(audio)) / sr
    plt.plot(time, audio, color=color, linewidth=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(title)
    plt.xlim(0, time[-1])
    plt.ylim(-1.1, 1.1)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_waveform_detail(audio, sr, title="Waveform Detail", n_cycles=4, freq=440, figsize=(12, 3)):
    """Plot a few cycles of a waveform to see the shape."""
    samples_per_cycle = int(sr / freq)
    n_samples = samples_per_cycle * n_cycles
    
    plt.figure(figsize=figsize)
    time = np.arange(n_samples) / sr * 1000  # Convert to ms
    plt.plot(time, audio[:n_samples], color='#2563eb', linewidth=1.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('Amplitude')
    plt.title(title)
    plt.ylim(-1.1, 1.1)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_spectrum(audio, sr, title="Frequency Spectrum", figsize=(12, 3)):
    """Plot the frequency spectrum."""
    n = len(audio)
    fft = np.abs(np.fft.rfft(audio))
    freqs = np.fft.rfftfreq(n, 1/sr)
    fft_db = 20 * np.log10(fft + 1e-10)
    
    plt.figure(figsize=figsize)
    plt.plot(freqs, fft_db, color='#2563eb', linewidth=0.5)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.title(title)
    plt.xlim(20, 15000)
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 1. Subtractive Synthesis

Classic analog-style synthesis with oscillators, filters, and envelopes.

In [ ]:
from audio_dsp.synth import SubtractiveSynth

synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)

# Compare different waveforms
waveforms = ["sine", "saw", "square", "triangle"]
colors = ['#2563eb', '#dc2626', '#16a34a', '#ca8a04']

fig, axes = plt.subplots(2, 2, figsize=(14, 6))
axes = axes.flatten()

for i, wave in enumerate(waveforms):
    synth.osc_wave = wave
    synth.filter_cutoff = 5000  # High cutoff to hear raw waveform
    audio = synth.synthesize(freq=440, duration=0.5)
    
    # Plot a few cycles
    samples_per_cycle = int(SAMPLE_RATE / 440)
    n_samples = samples_per_cycle * 3
    time = np.arange(n_samples) / SAMPLE_RATE * 1000
    
    axes[i].plot(time, audio[:n_samples], color=colors[i], linewidth=2)
    axes[i].set_title(f"{wave.capitalize()} Wave")
    axes[i].set_xlabel('Time (ms)')
    axes[i].set_ylabel('Amplitude')
    axes[i].set_ylim(-1.1, 1.1)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Play each waveform
for wave in waveforms:
    synth.osc_wave = wave
    synth.filter_cutoff = 2000
    audio = synth.synthesize(freq=220, duration=1.0)
    print(f"{wave.capitalize()} wave:")
    display(Audio(audio, rate=SAMPLE_RATE))

In [ ]:
# Classic bass sound
bass_synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
bass_synth.osc_wave = "saw"
bass_synth.filter_cutoff = 400
bass_synth.filter_resonance = 2.5
bass_synth.attack = 0.005
bass_synth.decay = 0.2
bass_synth.sustain = 0.4
bass_synth.release = 0.1

# Bass line
bass_notes = [55, 55, 73.4, 82.4, 55, 55, 82.4, 73.4]  # A1, A1, D2, E2...
bass_line = np.concatenate([bass_synth.synthesize(f, 0.25) for f in bass_notes])

plot_waveform(bass_line, SAMPLE_RATE, "Bass Line")
plot_spectrum(bass_line, SAMPLE_RATE, "Bass Line Spectrum")

print("Analog-style bass:")
Audio(bass_line, rate=SAMPLE_RATE)

In [ ]:
# Pad sound with slow attack
pad_synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
pad_synth.osc_wave = "triangle"
pad_synth.filter_cutoff = 1500
pad_synth.filter_resonance = 1.0
pad_synth.attack = 0.5
pad_synth.decay = 0.3
pad_synth.sustain = 0.8
pad_synth.release = 1.0

# Play a chord
chord_freqs = [261.6, 329.6, 392.0]  # C major
chord = sum(pad_synth.synthesize(f, 3.0) for f in chord_freqs) / len(chord_freqs)

plot_waveform(chord, SAMPLE_RATE, "Pad Chord (C Major)")

print("Pad chord:")
Audio(chord, rate=SAMPLE_RATE)

## 2. FM Synthesis (DX7-style)

Frequency modulation synthesis with 4 operators and multiple algorithms.

In [ ]:
from audio_dsp.synth import DX7FMSynth

fm_synth = DX7FMSynth(sample_rate=SAMPLE_RATE)

# Electric piano sound
fm_synth.algorithm = 1
fm_synth.ratios = [1.0, 1.0, 1.0, 1.0]
fm_synth.mod_indices = [0.5, 1.0, 0.3, 0.0]

epiano = fm_synth.synthesize(freq=440, duration=2.0)

plot_waveform(epiano, SAMPLE_RATE, "FM Electric Piano")
plot_spectrum(epiano, SAMPLE_RATE, "FM E-Piano Spectrum")

print("FM Electric Piano:")
Audio(epiano, rate=SAMPLE_RATE)

In [ ]:
# Metallic bell sound
bell_synth = DX7FMSynth(sample_rate=SAMPLE_RATE)
bell_synth.algorithm = 2
bell_synth.ratios = [1.0, 2.0, 3.5, 4.2]  # Inharmonic ratios for bell-like timbre
bell_synth.mod_indices = [2.0, 1.5, 1.0, 0.5]

bell = bell_synth.synthesize(freq=880, duration=3.0)

plot_waveform(bell, SAMPLE_RATE, "FM Bell")
plot_spectrum(bell, SAMPLE_RATE, "FM Bell Spectrum (Inharmonic Partials)")

print("FM Bell:")
Audio(bell, rate=SAMPLE_RATE)

In [ ]:
# Compare different algorithms
fig, axes = plt.subplots(5, 1, figsize=(12, 10))

for i, algo in enumerate(range(1, 6)):
    fm_synth.algorithm = algo
    fm_synth.ratios = [1.0, 2.0, 3.0, 4.0]
    fm_synth.mod_indices = [1.0, 0.8, 0.6, 0.4]
    sound = fm_synth.synthesize(freq=330, duration=1.5)
    
    time = np.arange(len(sound)) / SAMPLE_RATE
    axes[i].plot(time, sound, color='#2563eb', linewidth=0.5)
    axes[i].set_ylabel(f'Algo {algo}')
    axes[i].set_ylim(-1.1, 1.1)
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (s)')
plt.suptitle('FM Algorithm Comparison')
plt.tight_layout()
plt.show()

# Play each algorithm
for algo in range(1, 6):
    fm_synth.algorithm = algo
    sound = fm_synth.synthesize(freq=330, duration=1.5)
    print(f"Algorithm {algo}:")
    display(Audio(sound, rate=SAMPLE_RATE))

## 3. Chip Tune / 8-bit Sounds

Retro video game style sound generation.

In [ ]:
from audio_dsp.synth.chip_tone import (
    generate_kick,
    generate_snare,
    generate_cymbal,
    generate_blip,
    generate_laser,
    generate_powerup,
    generate_explosion
)

# Generate and visualize each sound
sounds = {
    "Kick": generate_kick(),
    "Snare": generate_snare(),
    "Hi-hat": generate_cymbal(),
    "Blip": generate_blip(),
    "Laser": generate_laser(),
    "Power-up": generate_powerup(),
    "Explosion": generate_explosion()
}

fig, axes = plt.subplots(len(sounds), 1, figsize=(12, 12))
colors = ['#dc2626', '#f97316', '#eab308', '#22c55e', '#06b6d4', '#8b5cf6', '#ec4899']

for i, (name, sound) in enumerate(sounds.items()):
    time = np.arange(len(sound)) / SAMPLE_RATE * 1000  # ms
    axes[i].plot(time, sound, color=colors[i], linewidth=0.8)
    axes[i].set_ylabel(name)
    axes[i].set_ylim(-1.1, 1.1)
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (ms)')
plt.suptitle('8-bit Sound Library')
plt.tight_layout()
plt.show()

# Play each sound
for name, sound in sounds.items():
    print(f"{name}:")
    display(Audio(sound, rate=SAMPLE_RATE))

In [ ]:
# Create a simple 8-bit drum pattern
kick = generate_kick()
snare = generate_snare()
hat = generate_cymbal()

# 16th note step length at 120 BPM
step_len = int(SAMPLE_RATE * 0.125)

def pad_sound(sound, length):
    if len(sound) >= length:
        return sound[:length]
    return np.pad(sound, (0, length - len(sound)))

kick_step = pad_sound(kick, step_len)
snare_step = pad_sound(snare, step_len)
hat_step = pad_sound(hat, step_len)

# Pattern: K---S---K---S--- with hats on 8ths
pattern = []
for i in range(16):
    step = np.zeros(step_len)
    if i in [0, 8]:  # Kick
        step += kick_step
    if i in [4, 12]:  # Snare
        step += snare_step
    if i % 2 == 0:  # Hats
        step += hat_step * 0.5
    pattern.append(step)

drum_loop = np.concatenate(pattern)
drum_loop = np.clip(drum_loop, -1, 1)

plot_waveform(drum_loop, SAMPLE_RATE, "8-bit Drum Pattern")

print("8-bit drum pattern:")
Audio(drum_loop, rate=SAMPLE_RATE)

## 4. Karplus-Strong Plucked Strings

Physical modeling synthesis for realistic plucked string sounds.

In [ ]:
from audio_dsp.synth import PluckSynth

pluck = PluckSynth(sample_rate=SAMPLE_RATE)

# Single plucked note
note = pluck.synthesize(freq=220, duration=2.0)

plot_waveform(note, SAMPLE_RATE, "Plucked String (A3)")
plot_spectrum(note, SAMPLE_RATE, "Plucked String Spectrum")

print("Plucked string (A3):")
Audio(note, rate=SAMPLE_RATE)

In [ ]:
# Guitar-like arpeggio
# E minor arpeggio: E2, G2, B2, E3, G3, B3, E4
arpeggio_freqs = [82.4, 98.0, 123.5, 164.8, 196.0, 246.9, 329.6]

arpeggio = []
for i, freq in enumerate(arpeggio_freqs):
    note = pluck.synthesize(freq=freq, duration=0.8)
    delay = np.zeros(int(i * 0.15 * SAMPLE_RATE))
    arpeggio.append(np.concatenate([delay, note]))

# Mix all notes together
max_len = max(len(a) for a in arpeggio)
mixed = np.zeros(max_len)
for a in arpeggio:
    mixed[:len(a)] += a

mixed = mixed / np.max(np.abs(mixed)) * 0.9

plot_waveform(mixed, SAMPLE_RATE, "Plucked Arpeggio (E minor)")

print("Plucked arpeggio:")
Audio(mixed, rate=SAMPLE_RATE)

## 5. Noise Generation

Various noise types for sound design and synthesis.

In [ ]:
from audio_dsp.utils import (
    white_noise,
    pink_noise,
    brown_noise,
    blue_noise,
    perlin_noise
)

duration = 2.0
n_samples = int(duration * SAMPLE_RATE)

noise_types = {
    "White Noise": white_noise(n_samples),
    "Pink Noise": pink_noise(n_samples),
    "Brown Noise": brown_noise(n_samples),
    "Blue Noise": blue_noise(n_samples),
    "Perlin Noise": perlin_noise(n_samples, scale=0.01)
}

# Visualize noise spectrums
fig, axes = plt.subplots(len(noise_types), 2, figsize=(14, 12))
colors = ['#6b7280', '#ec4899', '#92400e', '#3b82f6', '#22c55e']

for i, (name, noise) in enumerate(noise_types.items()):
    # Normalize
    noise = noise / np.max(np.abs(noise)) * 0.5
    noise_types[name] = noise  # Update for playback
    
    # Waveform (first 5000 samples)
    time = np.arange(5000) / SAMPLE_RATE * 1000
    axes[i, 0].plot(time, noise[:5000], color=colors[i], linewidth=0.3)
    axes[i, 0].set_ylabel(name)
    axes[i, 0].set_ylim(-0.6, 0.6)
    axes[i, 0].grid(True, alpha=0.3)
    
    # Spectrum
    fft = np.abs(np.fft.rfft(noise))
    freqs = np.fft.rfftfreq(len(noise), 1/SAMPLE_RATE)
    fft_db = 20 * np.log10(fft + 1e-10)
    axes[i, 1].plot(freqs[1:], fft_db[1:], color=colors[i], linewidth=0.3)
    axes[i, 1].set_xlim(20, 20000)
    axes[i, 1].set_xscale('log')
    axes[i, 1].grid(True, alpha=0.3)

axes[0, 0].set_title('Waveform')
axes[0, 1].set_title('Spectrum')
axes[-1, 0].set_xlabel('Time (ms)')
axes[-1, 1].set_xlabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

# Play each noise type
for name, noise in noise_types.items():
    print(f"{name}:")
    display(Audio(noise, rate=SAMPLE_RATE))

## 6. Combining Synths

Layer multiple synthesis techniques for richer sounds.

In [ ]:
from audio_dsp.utils import normalize_audio
from audio_dsp.effects import filter_effect

freq = 220
duration = 2.0

# Layer 1: Warm saw
sub_synth = SubtractiveSynth(sample_rate=SAMPLE_RATE)
sub_synth.osc_wave = "saw"
sub_synth.filter_cutoff = 1000
sub_synth.attack = 0.1
sub_synth.release = 0.5
layer1 = sub_synth.synthesize(freq, duration)

# Layer 2: FM shimmer (octave up)
fm = DX7FMSynth(sample_rate=SAMPLE_RATE)
fm.algorithm = 1
fm.ratios = [1.0, 2.01, 3.0, 4.0]  # Slight detune for movement
fm.mod_indices = [0.3, 0.2, 0.1, 0.0]
layer2 = fm.synthesize(freq * 2, duration) * 0.3

# Layer 3: Filtered noise for texture
noise = white_noise(int(duration * SAMPLE_RATE)) * 0.1
layer3 = filter_effect(noise, SAMPLE_RATE, cutoff=2000, resonance=2.0)
envelope = np.linspace(1, 0, len(layer3)) ** 2
layer3 = layer3 * envelope

# Visualize layers
fig, axes = plt.subplots(4, 1, figsize=(12, 10))

time = np.arange(len(layer1)) / SAMPLE_RATE

axes[0].plot(time, layer1, color='#2563eb', linewidth=0.5)
axes[0].set_ylabel('Layer 1\n(Saw)')
axes[0].set_ylim(-1.1, 1.1)
axes[0].grid(True, alpha=0.3)

axes[1].plot(time, layer2, color='#dc2626', linewidth=0.5)
axes[1].set_ylabel('Layer 2\n(FM)')
axes[1].set_ylim(-0.5, 0.5)
axes[1].grid(True, alpha=0.3)

axes[2].plot(time, layer3, color='#16a34a', linewidth=0.5)
axes[2].set_ylabel('Layer 3\n(Noise)')
axes[2].set_ylim(-0.2, 0.2)
axes[2].grid(True, alpha=0.3)

# Mix layers
mixed = layer1 * 0.6 + layer2 * 0.25 + layer3 * 0.15
mixed = normalize_audio(mixed, peak=0.9)

axes[3].plot(time, mixed, color='#8b5cf6', linewidth=0.5)
axes[3].set_ylabel('Mixed')
axes[3].set_xlabel('Time (s)')
axes[3].set_ylim(-1.1, 1.1)
axes[3].grid(True, alpha=0.3)

plt.suptitle('Layered Synthesis')
plt.tight_layout()
plt.show()

print("Layered synth (Subtractive + FM + Noise):")
Audio(mixed, rate=SAMPLE_RATE)